## Importing Libraries

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer

In [ ]:
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import LabelEncoder

## Importing data

## Prediction

In [ ]:
# Filling data path
# train_data_path = "./data/train.csv"

# # reading CSV file
# df = pd.read_csv(train_data_path)
# df.drop_duplicates(inplace=True)


## Submission
##### Splitting feature and target data and concatenating the Dataframes

In [ ]:
# Filling data path
train_data_path = "./data/train.csv"
test_data_path = "./data/test.csv"

# reading CSV file
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)

# Creating column to differentiate df_test and df_train
df_train['train_dummy'] = 1
df_test['train_dummy'] = 0

df = pd.concat([df_train, df_test], axis=0).sort_values("id").reset_index()
print(df.shape)


(10000, 17)


## Data cleaning
- Dropping columns
- Creating Floor 1 Dummy
- Removing outliers from the num_rooms and square_meters

In [ ]:
# Dropping orientation (argue saying that this is hardly inputer and has a 30% of missing data) 
df.drop('orientation', axis=1, inplace=True)

# Creating floor variable
df[['floor', 'door_num']] = df['door'].str.split('-', n=1, expand=True)
df['floor'] = df['floor'].str[0]
df["floor"] = pd.to_numeric(df["floor"])

# Dropping door and door_num columns (justify: not influential)
df.drop(['door', 'door_num'], axis=1, inplace=True)

# Replacing the outliers with NaN in the number of rooms (justify cutoff value: outliers are very high above 10)
df['num_rooms'] = df['num_rooms'].apply(lambda x: x if x<10 else np.nan)

# Replacing the values of square metres < 40 with NaN (change the cutoff value and see the results)
df.loc[df['square_meters'] < 0, 'square_meters'] = np.nan

In [ ]:
print(df.columns)

Index(['index', 'id', 'num_rooms', 'num_baths', 'square_meters', 'year_built',
       'is_furnished', 'has_pool', 'neighborhood', 'num_crimes', 'has_ac',
       'accepts_pets', 'num_supermarkets', 'price', 'train_dummy', 'floor'],
      dtype='object')


## Standardizing

In [ ]:
# Standardization
to_standardize = ['num_rooms', 'num_baths', 'square_meters', 'year_built', 'neighborhood', 'num_crimes', 'price', 'floor']

for i in to_standardize:
    df[i] = (df[i] - np.mean(df[i])) / np.std(df[i])